In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-02-07 22:28:00,061 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-07 22:28:00,062 - settings - Setting database.user to celiib
INFO - 2021-02-07 22:28:00,063 - settings - Setting database.password to newceliipass
INFO - 2021-02-07 22:28:00,067 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-07 22:28:00,068 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-07 22:28:00,078 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-07 22:28:00,367 - settings - Setting enable_python_native_blobs to True


In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = True

# Debugging the contains method

In [4]:
import system_utils as su

In [5]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-07 22:28:00,474 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-07 22:28:00,476 - settings - Setting database.user to celiib
INFO - 2021-02-07 22:28:00,477 - settings - Setting database.password to newceliipass
INFO - 2021-02-07 22:28:00,479 - settings - Setting enable_python_native_blobs to True


Sleeping 122 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-02-07 22:28:00,775 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 7661


In [8]:
# minnie.DecompositionSplit.drop()
# schema.external['decomposition'].delete(delete_external_files=True)

In [9]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0"
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134221889045,0,0.25,30.00,5,0,1,=BLOB=,12285,26788,0,1,1,0,1,1,3,3,7497.255110138911,7497.255110138911,942.191,298.182,298.182,1077.139,1077.139,1,0.00013338214924121366,0.3333333333333333,7497.255,3,0.00013338215120067278,0.333,55229792.033337906,55229792.033337906,7366.66836355187,7366.668471772389,18409930.677779302,16.8753
864691134589113354,0,0.25,30.00,0,0,1,=BLOB=,16959,35490,0,1,1,0,1,1,1,1,6500.555290747829,6500.555290747829,6500.555,81.132,81.132,81.132,81.132,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.4638
864691134651920138,0,0.25,30.00,0,0,1,=BLOB=,11097,23083,0,1,1,0,1,1,3,3,12121.237085842979,12121.237085842979,3524.0,38.678,38.678,89.187,89.187,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.5987
864691134668951908,0,0.25,30.00,0,0,1,=BLOB=,10027,21487,0,1,1,0,1,1,3,3,30717.021602829212,30717.021602829212,11655.34,55.344,55.344,99.027,99.027,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.6528
864691134691461988,0,0.25,30.00,0,0,1,=BLOB=,30322,61377,0,1,1,0,1,1,20,20,271693.69158879406,271693.69158879406,12486.756,97.909,97.909,200.711,200.711,6,2.2083692723646103e-05,0.3,92634.578,6,6.477063024996995e-05,1.0,907690952.2834566,65986064.39167149,3340.861346376929,9798.619175265812,151281825.3805761,43.9805
864691134743737238,0,0.25,30.00,0,0,1,=BLOB=,23704,48650,0,2,2,0,1,2,7,4,41164.30323551219,31508.63736225171,5515.413,104.17,104.17,415.659,409.237,4,9.717157064738618e-05,0.5714285714285714,8608.626,2,0.0004646502240891868,2.0,217755544.6000223,24646700.83832677,5289.912071490279,25295.04064876582,108877772.30001114,51.37
864691134765143804,0,0.25,30.00,0,0,1,=BLOB=,44570,90882,0,1,1,1,2,1,1,1,28585.105732166514,28585.105732166514,28585.106,1726.047,1726.047,1726.047,1726.047,4,0.00013993301397863443,4.0,28585.106,1,0.00013993301266750595,4.0,174766472.25666714,13946453.78167041,6113.899801322207,6113.899744036882,174766472

In [10]:
import numpy as np
import time
import proofreading_utils as pru

split_version = 0
split_version = 1 #fixed the problem with split from suggestions

verbose = True

@schema
class DecompositionSplit(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index: tinyint unsigned  #the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    split_version: tinyint unsigned  #the version of the splitting algorithm used
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    n_splits             : int unsigned                 # the number of cuts required to help split the neuron
    split_success        : tinyint unsigned             # the successfulness of the splitting
    
    n_error_limbs_cancelled : tinyint unsigned     # number of limbs that couldn't be resolved and cancelled out        
    n_same_soma_limbs_cancelled : tinyint unsigned     # number of same soma touching limbs that couldn't be resolved and cancelled out
    n_multi_soma_limbs_cancelled : tinyint unsigned     # number of multi soma touching limbs that couldn't be resolved and cancelled out        
    
    error_imbs_cancelled_area=NULL : double            # the total area (in microns^2) of the limbs that was cancelled out because touching the same soma multiple times or multiple somas
    error_imbs_cancelled_skeletal_length = NULL : double #the total skeletal length (in microns) of the limbs that were called out because could not be resolved
    
    split_results: longblob #will store the results of how to split the limbs of neuron objects from original neuron
    decomposition: <decomposition>
    
    
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    max_soma_n_faces:  int unsigned                 # The largest number of faces of the somas
    max_soma_volume:  int unsigned                 # The largest volume of the somas the (volume in billions (10*9 nm^3))
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """
                             
    
    ''' Old keysource used for inhibitory excitatory check
    classified_table = (minnie.BaylorManualCellType() &
                        'nucleus_version=3' & 
                        "(cell_type = 'excitatory') or  (cell_type = 'inhibitory')")
    
    key_source = ((minnie.Decomposition & 
                (minnie.NeuronSplitSuggestions.proj()) & 
                (classified_table.proj()) 
                & f"n_somas<{max_n_somas}" & "n_error_limbs>0"))'''
    
    # This keysource acounts that you could have more than 1 possible soma but not a significant limb connecting them (no error limbs)
    key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & "segment_id=864691135883900016"
    
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decomposed neurong object from Decomposition table
        3) Run the multi_soma split suggestions algorithm
        4) Get the number of splits required for this neuron
        5) Split the neuron into a list of neuron objects
        6) For each neuron object in the list:
        - get the number of errored limbs (to indicate the success type)
        - Change the description to include the multiplicity
        - Compute the information on the largest soma faces and volume
        - Save the neuron object to the external
        - Add the new write key to a list to commit 
        7) Write all of the keys 
        """
        
        whole_pass_time = time.time()
        
        
        
        # 1) Get the segment id from the key
        segment_id = key["segment_id"]
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        
        
        # 2) Get the decomposed neuron object from Decomposition table and the split suggestions
        neuron_obj = (minnie.Decomposition & key).fetch1("decomposition")
        
        """ Old way that downloaded from another table
        # 3) Retrieve the multi soma suggestions
        split_results = (minnie.NeuronSplitSuggestions & key).fetch1("split_results")
        """
        #3) Calculated the split results
        split_results = pru.multi_soma_split_suggestions(neuron_obj,plot_intermediates=False)
        
        # 4) Get the number of splits required for this neuron
        n_paths_cut = pru.get_n_paths_cut(split_results)

        if verbose:
            print(f"n_paths_cut = {n_paths_cut}")
            
            
        # 5) Split the neuron into a list of neuron objects
        (neuron_list,
        neuron_list_errored_limbs_area,
         neuron_list_errored_limbs_skeletal_length,
        neuron_list_n_multi_soma_errors,
        neuron_list_n_same_soma_errors) = pru.split_neuron(neuron_obj,
                        limb_results=split_results,
                                       verbose=verbose,
                                        return_error_info=True
                                            )
        
        print(f"neuron_list = {neuron_list}")
        print(f"neuron_list_errored_limbs_area = {neuron_list_errored_limbs_area}")
        print(f"neuron_list_n_multi_soma_errors = {neuron_list_n_multi_soma_errors}")
        print(f"neuron_list_n_same_soma_errors = {neuron_list_n_same_soma_errors}")
        
        
        if verbose:
            print(f"Number of neurons: {len(neuron_list)}")
        
        neuron_entries = []
        for neuron_idx in range(len(neuron_list)):
            
            """
            # 6) For each neuron object in the list:
            # - get the number of errored limbs (to indicate the success type)
            # - Compute the information on the largest soma faces and volume
            # - Save the neuron object to the external
            # - Add the new write key to a list to commit 
            """
            n = neuron_list[neuron_idx]
            
            error_imbs_cancelled_area = neuron_list_errored_limbs_area[neuron_idx]
            error_imbs_cancelled_skeletal_length = neuron_list_errored_limbs_skeletal_length[neuron_idx]
            n_multi_soma_limbs_cancelled = neuron_list_n_multi_soma_errors[neuron_idx]
            n_same_soma_limbs_cancelled = neuron_list_n_same_soma_errors[neuron_idx]
            
            
            #for n in neuron_list:
            #     nviz.visualize_neuron(n,
            #                          limb_branch_dict="all")

            # - get the number of errored limbs (to indicate the success type)
            if n.n_error_limbs == 0:
                split_success = 0
            elif n.multi_soma_touching_limbs == 0:
                split_successs = 1
            elif n.same_soma_multi_touching_limbs == 0:
                split_success = 2
            else:
                split_success = 3

            if verbose:
                print(f"split_success = {split_success}")

            # - Compute the information on the largest soma faces and volume
            soma_volumes = [n[k].volume/1000000000 for k in n.get_soma_node_names()] 
            soma_n_faces = [len(n[k].mesh.faces) for k in n.get_soma_node_names()] 

            largest_n_faces = np.max(soma_n_faces)
            largest_volume = np.max(soma_volumes)

            if verbose:
                print(f"largest_n_faces = {largest_n_faces}")
                print(f"largest_volume = {largest_volume}")

            if "split" not in n.description:
                n.description += "_soma_0_split"
                
            #6) Save the file in a certain location
            if True:
                save_time = time.time()
                ret_file_path = n.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                                  return_file_path=True,
                                                 export_mesh=False,
                                                 suppress_output=True)

                ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
                print(f"Save time = {time.time() - save_time}")
            else:
                print("Storing a dummy value for neuron")
                ret_file_path_str = "dummy"



            #7) Pass stats and file location to insert
            new_key = dict(key,
                           split_index = neuron_idx,
                           split_version = split_version,
                           
                           multiplicity=len(neuron_list),

                           n_splits = n_paths_cut,
                           split_success = split_success,
                           
                           n_error_limbs_cancelled = len(error_imbs_cancelled_area),
                           
                           n_multi_soma_limbs_cancelled =n_multi_soma_limbs_cancelled,
                           n_same_soma_limbs_cancelled = n_same_soma_limbs_cancelled,
                           error_imbs_cancelled_area = np.round(np.sum(error_imbs_cancelled_area),4),
                           error_imbs_cancelled_skeletal_length = np.round(np.sum(error_imbs_cancelled_skeletal_length)/1000,4),
                           
                           split_results=split_results,

                           max_soma_n_faces = largest_n_faces,
                           max_soma_volume = largest_volume,


                           decomposition=ret_file_path_str,
                           n_vertices=len(n.mesh.vertices),
                           n_faces=len(n.mesh.faces),
                           run_time=np.round(time.time() - whole_pass_time,4)
                          )

            stats_dict = n.neuron_stats()
            new_key.update(stats_dict)


            neuron_entries.append(new_key)

        
        self.insert(neuron_entries, allow_direct_insert=True, skip_duplicates=True)
        

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - whole_pass_time} ------")
    

# Running the Populate

In [11]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition_split,009e6c8328e3b3bff056b8ce9c524fbc,error,=BLOB=,IndexError: index 10098 is out of bounds for axis 0 with size 10098,=BLOB=,celiib@10.28.0.150,at-node23,1,113356,2021-02-03 12:09:01
__decomposition_split,0193b0ceee3dbfb1a0129a33f51b25cb,error,=BLOB=,"Exception: piece connectivity ([0, 1, 2, 3, 4, 5, 6]) not match limb neighbors ([])",=BLOB=,celiib@10.28.0.177,at-node50,1,112321,2021-02-03 07:44:29
__decomposition_split,01c74daca008117152a53ca441a71306,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.144,at-node17,1,112052,2021-02-03 07:01:44
__decomposition_split,01cf4a0f25efca5ad38c441aab5ecba5,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.145,at-node18,1,112101,2021-02-03 13:05:29
__decomposition_split,02ec8e9238fa0fc2bb32345caa08c45c,error,=BLOB=,IndexError: index 124610 is out of bounds for axis 0 with size 124610,=BLOB=,celiib@10.28.0.161,at-node34,1,112211,2021-02-03 13:52:45
__decomposition_split,02ee19ad9a68299a4c602e6af5edd3e2,error,=BLOB=,IndexError: list index out of range,=BLOB=,celiib@10.66.4.64,jr-compute001,1,112199,2021-02-03 06:58:26
__decomposition_split,03333b84aebe25314db2b2eef70f6e2e,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.147,at-node20,1,112086,2021-02-03 04:15:53
__decomposition_split,0376e2e8d41a2fc9bbea42e30bcf3cca,error,=BLOB=,SystemExit: SIGTERM received,=BLOB=,celiib@10.28.0.160,at-node33,1,112089,2021-02-03 14:53:26
__decomposition_split,03b2645c30ea2d582e15c9b5442fcebe,error,=BLOB=,IndexError: index 4524 is out of bounds for axis 0 with size 4524,=BLOB=,celiib@10.28.0.177,at-node50,1,112324,2021-02-03 08:45:28
__decomposition_split,0526605b148dd5629d420781702b32f7,error,=BLOB=,"Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)",=BLOB=,celiib@10.28.0.81,at-compute003,1,112252,2021-02-03 04:59:41


In [13]:
import time
import random
pru = reload(pru)
nru = reload(nru)
import neuron
neuron = reload(neuron)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=True, order="random")
else:
    DecompositionSplit.populate(reserve_jobs=True, suppress_errors=False,order="random")
print('Populate Done')

print(f"Total time for DecompositionSplit populate = {time.time() - start_time}")

Populate Started



---- Working on Neuron 864691135883900016 ----
Decompressing Neuron in minimal output mode...please wait
Skipping endnode 44 because skeletal distance was 846.0741971544592 and threshold was 2500
More than one common endpoint so just choosing the first
More than one common endpoint so just choosing the first
Skipping endnode 44.0 because skeletal distance was 846.0741971544592 and threshold was 2500
n_paths_cut = 1
using precomputed split suggestions
More than one common endpoint so just choosing the first
More than one common endpoint so just choosing the first
More than one common endpoint so just choosing the first
More than one common endpoint so just choosing the first


---Working on Splitting Limb 0 with 6 components----


----Working on seperate_graph 0----


---------Working on soma_idx = 0, soma_group_idx 0, endpt = [1407960.  441230. 1067270.]---------
Starting_edge inside branches_to_conept = [[1407960.  441230. 1067270.]
 [1409120.  436467. 1067110.]]
A

Exception: after loop in directed concept graph, not all nodes have incoming edges (except starter node)